<a href="https://www.kaggle.com/code/trevorglenn/simple-tfregression?scriptVersionId=178450511" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [2]:
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.metrics import r2_score

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler, MinMaxScaler

2024-05-18 22:35:52.446026: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 22:35:52.446154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 22:35:52.606906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_df = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [4]:
train_df.shape

(1117957, 22)

In [5]:
test_df.shape

(745305, 21)

In [6]:
train_df.drop('id', axis=1)

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,3,3,4,10,4,5,5,7,10,4,...,7,8,7,2,2,1,4,6,4,0.495
1117953,2,2,4,3,9,5,8,1,3,5,...,9,4,4,3,7,4,9,4,5,0.480
1117954,7,3,9,4,6,5,9,1,3,4,...,5,5,5,5,6,5,5,2,4,0.485
1117955,7,3,3,7,5,2,3,4,6,4,...,6,8,5,3,4,6,7,6,4,0.495


In [7]:
test_df.columns

Index(['id', 'MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors'],
      dtype='object')

In [8]:
train_features = train_df.drop('FloodProbability', axis=1)
train_target = train_df['FloodProbability']

scalable_features = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']

test_id = test_df[['id']]

In [9]:
mean = train_features.mean(axis=0)
train_features -= mean
std = train_features.std(axis=0)
train_features /= std

test_df -= mean
test_df /= std

In [10]:
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.25, random_state=24)

In [12]:
def build_model():
    model = keras.Sequential([
        layers.InputLayer(input_shape=(X_train.shape[1], )),
        layers.Dense(8, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(64, activation='relu'),
        #layers.Dense(128, activation='relu'),
        #layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [13]:
model = build_model()
es = EarlyStopping(monitor='val_mae', mode='min', verbose=1, patience=15)
model.fit(X_train, y_train, epochs=100, batch_size=100, verbose=1, validation_split=0.1, callbacks=[es])
model.summary()
y_pred_model = model.predict(X_test)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.0020 - mae: 0.0274 - val_loss: 3.8984e-04 - val_mae: 0.0153
Epoch 2/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 3.9945e-04 - mae: 0.0157 - val_loss: 3.8517e-04 - val_mae: 0.0150
Epoch 3/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 3.8365e-04 - mae: 0.0154 - val_loss: 3.6808e-04 - val_mae: 0.0153
Epoch 4/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 3.7774e-04 - mae: 0.0153 - val_loss: 3.6303e-04 - val_mae: 0.0149
Epoch 5/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 3.7303e-04 - mae: 0.0152 - val_loss: 3.6052e-04 - val_mae: 0.0150
Epoch 6/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 3.7152e-04 - mae: 0.0151 - val_loss: 3.7163e-04 - val_mae: 0.0148
Epoch 7/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 3.6991e-04 - mae: 0.0151 - val_loss: 3.6879e-04 - val_mae: 0.0148
Epoch 8/100
7547/7547 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 3.6821e-04 - mae: 0.0151 - val

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,588 (60.89 KB)

 Trainable params: 7,793 (30.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,795 (30.45 KB)

8735/8735 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step


In [14]:
r2_score(y_test, y_pred_model)

0.8597531791385702

In [15]:
y_stacking = model.predict(test_df)

df_y_stacking_out = test_id
df_y_stacking_out['FloodProbability'] = y_stacking

df_y_stacking_out.to_csv('submission.csv', index=False)

23291/23291 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step


/tmp/ipykernel_33/1709693510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_y_stacking_out['FloodProbability'] = y_stacking


In [16]:
df_y_stacking_out.head()

,id,FloodProbability
0,1117957,0.579123
1,1117958,0.455389
2,1117959,0.454216
3,1117960,0.474867
4,1117961,0.474898
